# Video next to Skeleton64 next to Skeleton20  

In [1]:
%reset -f
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

In [2]:
import pandas as pd
import os
import numpy as np

ANNOTATIONS_PATH = 'data'
REPROCESSED_PATH = ANNOTATIONS_PATH+'/reprocessed'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#annotations_file = ANNOTATIONS_PATH+'/annotationsVSingle.txt'
annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'

with open(annotations_file,'r') as file:
    annotations = file.read().splitlines()[1:]

sampleAnno = 1

if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
    folder, trial, fragId, _, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
else:
    folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]

reprocessed_file = os.path.join(REPROCESSED_PATH, folder, trial+'_frag'+fragId+'.csv')
reducedMarkersTable = pd.read_csv(reprocessed_file)
raw_file = os.path.join(RAW_PATH, folder, trial+'.tsv')
with open(raw_file,'r') as file:
    numMarkers, columnNames, readerBuffer = custom_header_reader(file)
    fullMarkersTable = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
fullMarkersTable.iloc[:,3:] = (fullMarkersTable.iloc[:,3:] - fullMarkersTable.iloc[:,3:].min(axis=None))/(fullMarkersTable.iloc[:,3:].max(axis=None)-fullMarkersTable.iloc[:,3:].min(axis=None))
fullMarkersTable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6196 entries, 0 to 6195
Columns: 195 entries, Frame to LPLM Z
dtypes: float64(193), int64(1), object(1)
memory usage: 9.2+ MB


## Video face censoring
TODO improve

In [3]:
import cv2
import numpy as np
from tqdm.notebook import tqdm
import os

# Load a pre-trained face detection model (e.g., Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
half_body_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_upperbody.xml')


video_path = '/'.join(os.getcwd().split('/')[:-2]+['docs','presentation','resources','_'.join([folder,trial])+'.mp4'])

vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(100)+'fps.mp4')

# Get video details
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define a function to censor faces
def censor_faces(frame,i):
    if i < 5800:
        frame_crop = frame[height//4:height//4+int(height/2.5),int(width/7.5):int(width/7.5)+int(width/2.1)]
        gray = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,scaleFactor=1.1, minNeighbors=5, minSize=(15, 15), maxSize=(40,40))
    else:
        frame_crop = frame[height//3:height//3+int(height/6),int(width/1.9):int(width/1.9)+int(width/6)]
        gray = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.03, minNeighbors=1, minSize=(30, 30), maxSize=(40,40))
    if not any(static_face_censor_delay) and len(faces) == 0:
        gray = cv2.addWeighted(gray, 3.1, np.zeros_like(gray), 0, 0)
        half_body = half_body_cascade.detectMultiScale(gray, scaleFactor=1.01, minNeighbors=5, minSize=(120, 120), maxSize=(200,200))
        return *apply_censoring(frame,half_body,is_body=True), half_body
    return *apply_censoring(frame,faces,i=i), faces

def apply_censoring(frame, bb, i=-1, is_body=False):
    for (x, y, w, h) in bb:
        # Apply censorship here, e.g., pixelation
        if is_body:
            x += int(width/7.5) + 30 ; y += height//4 + 40
            w = 60; h = 50
        else:
            if i < 5800:
                enlarger = 4
                x += int(width/7.5) - enlarger; y += height//4 - enlarger
                w += 4*enlarger; h += 4*enlarger
            else:
                enlarger = 10
                x += int(width/1.9)-enlarger ; y += height//3-enlarger
                w += 2*enlarger; h += 2*enlarger
        roi = frame[y:y+h,x:x+w]
        roi = cv2.GaussianBlur(roi, (23, 23), 9)
        frame[y:y+h, x:x+w] = roi
    return frame, is_body

static_face_window = 400
static_face_censor_delay = np.zeros(static_face_window, bool)
moving_face_window = 300
moving_face_censor_delay = np.zeros(moving_face_window, bool)

last_faces = None

if not os.path.isfile(video_path[:-4]+'_face_censored.mp4'):
    # Create an output video writer
    out = cv2.VideoWriter(video_path[:-4]+'_face_censored.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    _, frame = vidcap.read()
    faces = [[477, 75, 30, 30]]
    frame,_ = apply_censoring(frame, faces)
    censored = True
    is_body = False
    for i in tqdm(range(1,num_frames), 'Blurring face',miniters=20):
        if is_body:
            moving_face_censor_delay[i%moving_face_window] = censored
            static_face_censor_delay[i%static_face_window] = False
        else:
            static_face_censor_delay[i%static_face_window] = censored
            moving_face_censor_delay[i%moving_face_window] = False

        if censored:
            last_faces = faces
        if not censored and ((not is_body and any(static_face_censor_delay)) or (is_body and any(moving_face_censor_delay))):
            frame,_ = apply_censoring(frame, last_faces, is_body=is_body)
        #frame = cv2.rectangle(frame, (int(width/7.5), height//4), (int(width/7.5) + int(width/2.1), height//4 + int(height/2.5)), (255, 0, 0), 2)
        # Add the text to the image
        #frame = cv2.putText(frame, str(i), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        # Write the frame to the output video
        out.write(frame)

        _, frame = vidcap.read()
        
        # Apply censorship to the frame
        frame, is_body, faces = censor_faces(frame,i)
        censored = len(faces) > 0

    out.release()
vidcap.release()


In [4]:
import cv2
import os
from tqdm.notebook import tqdm
import numpy as np

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

# Initializing the subtractor
fgbg = cv2.createBackgroundSubtractorKNN(history=100,dist2Threshold=500)

input_video = cv2.VideoCapture('video.mp4')

frame_width = int(input_video.get(3))
frame_height = int(input_video.get(4))
fps = int(input_video.get(5))
num_frames = int(input_video.get(cv2.CAP_PROP_FRAME_COUNT))

if not os.path.isfile('videoNoBG.mp4'):
    output_video = cv2.VideoWriter('videoNoBG.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height), isColor=True)
    
    for i in tqdm(range(num_frames), 'Subtracting BG', miniters=40):
        ret, frame = input_video.read()
        
        # Apply the background subtractor KNN
        fgmask = fgbg.apply(frame)
        
        # Clean up the mask using morphological operations
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
        
        # Invert the mask to keep the foreground
        fg = cv2.bitwise_and(frame, frame, mask=fgmask)
        
        # Save the frame to the output video
        output_video.write(fg)
    
    output_video.release()
    
# Release the video objects
input_video.release()

Subtracting BG: 0it [00:00, ?it/s]

In [5]:
import xml.etree.ElementTree as ET

# Replace 'your_file.xml' with the actual XML file you want to read
xml_file = ANNOTATIONS_PATH+'/labels.xml'

# Parse the XML file
root = ET.parse(xml_file).getroot()

bones = []
max_indx = 0
# Explore the "Bone" elements
for bone in root.findall(".//Bone"):
    #print("Bone:")
    for index in bone.findall(".//Index"):
        low = index.get("Low")
        high = index.get("High")
        #print(f"  Index: Low={low}, High={high}")
        bones.append((int(low)-1,int(high)-1))
        if int(low) > max_indx or int(high) > max_indx:
            max_indx = max(int(low),int(high))

markers = []
# Explore the "Trajectory" elements
for trajectory in root.findall(".//Trajectory"):
    name = trajectory.find(".//Name").text
    #print(f"Trajectory: Name={name}")
    markers.append(name)

In [6]:
fullPosTableY = fullMarkersTable.iloc[:,3::3]
fullPosTableZ = fullMarkersTable.iloc[:,4::3]
fullPosTableX = fullMarkersTable.iloc[:,5::3]

fullPosTableX = fullPosTableX.rename(columns=dict(zip(list(fullPosTableX),[name.replace('Z','X').replace('Y','X') for name in list(fullPosTableX)])))
fullPosTableY = fullPosTableY.rename(columns=dict(zip(list(fullPosTableY),[name.replace('Z','Y').replace('X','Y') for name in list(fullPosTableY)])))
fullPosTableZ = fullPosTableZ.rename(columns=dict(zip(list(fullPosTableZ),[name.replace('X','Z').replace('Y','Z') for name in list(fullPosTableZ)])))

In [7]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


edges = np.array(bones)
adjacencyMatrix = np.zeros((max_indx,max_indx),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig = plt.figure(figsize=(960/100, 720/100), dpi=100)

# Add a 3D subplot with the specified position and size
ax = fig.add_axes([-0.05, -0.4, 1, 1.7], projection='3d')
ax.view_init(elev=10, azim=180)


minMax = np.zeros((2,3))
minMax[0,:] = [fullPosTableX.values.min(),fullPosTableY.values.min(),fullPosTableZ.values.min()]
minMax[1,:] = [fullPosTableX.values.max(),fullPosTableY.values.max(),fullPosTableZ.values.max()]

# Set appropriate axis limits
ax.set_xlim([minMax[0,0],minMax[1,0]])
ax.set_ylim([minMax[0,1],minMax[1,1]])
ax.set_zlim([minMax[0,2],minMax[1,2]])

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 1000
colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']
plt.rcParams['grid.color'] = 'white'
ax.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax.w_xaxis.pane.fill = False
ax.w_yaxis.pane.fill = False
ax.w_zaxis.pane.fill = True
ax.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax.w_xaxis.line.set_visible(False)
ax.w_yaxis.line.set_visible(True)
ax.w_zaxis.line.set_visible(False) 

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = fullPosTableX.iloc[val]
    filteredY = fullPosTableY.iloc[val]
    filteredZ = fullPosTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=10)

    # Add edges based on the weight matrix
    for i in range(max_indx):
        for j in range(i + 1, max_indx):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='k', linestyle='-', linewidth=0.7)

    ax.set_xlim([0,1])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([0,1])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    #ax.set_xlabel('X', fontsize=12)
    #ax.set_ylabel('Y', fontsize=12)

    #ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))
    #ax.grid(axis=['x','y'])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    #ax.set_xticklabels([])
    #ax.set_yticklabels([])
    #ax.set_zticklabels([])
    fig.canvas.draw_idle()
    
# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = fullPosTableX.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep <= maxValue:
            slider.set_val(slider.val + slider.valstep)
    elif event.button == 'up':
        if slider.val - slider.valstep >= 0:
            slider.set_val(slider.val - slider.valstep)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: <object object at 0x7fe0a6b816d0>


/tmp/ipykernel_3233/3347424033.py:38: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax.w_xaxis.pane.fill = False
/tmp/ipykernel_3233/3347424033.py:39: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax.w_yaxis.pane.fill = False
/tmp/ipykernel_3233/3347424033.py:40: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax.w_zaxis.pane.fill = True
/tmp/ipykernel_3233/3347424033.py:41: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_3233/3347424033.py:46: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax.w_xax

In [8]:
import cv2
import os

# Open the video file
video_path = '/'.join(os.getcwd().split('/')[:-2]+['docs','presentation','resources','_'.join([folder,trial])+'.mp4'])
print(video_path)
cap = cv2.VideoCapture(video_path)

# Get the frames per second (FPS)
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(fps, cap.get(cv2.CAP_PROP_FRAME_COUNT))
ret, prev_frame = cap.read()
ret= not os.path.isfile(video_path[:-4]+'_'+str(fps * 2)+'fps.mp4')
num_frames = 0
if ret:
    output = cv2.VideoWriter(video_path[:-4]+'_'+str(fps * 2)+'fps.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps * 2, (int(cap.get(3)), int(cap.get(4))))
    while True:
        if num_frames < len(fullMarkersTable):
            output.write(prev_frame)
            num_frames += 1
        ret, frame = cap.read()
        if not ret:
            break
        averaged_frame = cv2.addWeighted(prev_frame, 0.5, frame, 0.5, 0)
        if num_frames < len(fullMarkersTable):
            output.write(averaged_frame)
            num_frames += 1
        else: break
        prev_frame = frame
    output.release()
else: 
    vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(fps * 2)+'fps_540p.mp4')
    print(vidcap.get(cv2.CAP_PROP_FRAME_COUNT), vidcap.get(3),vidcap.get(4))
print(num_frames)

# Release the video file
cap.release()


/home/gagg/Desktop/Thesis/OoM-Thesis/docs/presentation/resources/2016-05-26_t_006.mp4
50 3201.0
6195.0 960.0 540.0
0


In [9]:
from matplotlib import pyplot as plt
%matplotlib

edges = np.array(bones)
fullAdjacencyMatrix = np.zeros((max_indx,max_indx),dtype=bool)
fullAdjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig_full = plt.figure(figsize=(960/100, 540/100), dpi=100)

# Add a 3D subplot with the specified position and size
ax_full = fig_full.add_axes([-0.3, -0.2, 1.6, 1.6], projection='3d')
ax_full.view_init(elev=10, azim=180)


fullMinMax = np.zeros((2,3))
fullMinMax[0,:] = [fullPosTableX.values.min(),fullPosTableY.values.min(),fullPosTableZ.values.min()]
fullMinMax[1,:] = [fullPosTableX.values.max(),fullPosTableY.values.max(),fullPosTableZ.values.max()]

# Set appropriate axis limits
ax_full.set_xlim([fullMinMax[0,0],fullMinMax[1,0]])
ax_full.set_ylim([fullMinMax[0,1],fullMinMax[1,1]])
ax_full.set_zlim([fullMinMax[0,2],fullMinMax[1,2]])


plt.rcParams['grid.color'] = 'white'
ax_full.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax_full.w_xaxis.pane.fill = False
ax_full.w_yaxis.pane.fill = False
ax_full.w_zaxis.pane.fill = True
ax_full.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax_full.xaxis.pane.set_edgecolor('w')
ax_full.yaxis.pane.set_edgecolor('w')
ax_full.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax_full.w_xaxis.line.set_visible(False)
ax_full.w_yaxis.line.set_visible(True)
ax_full.w_zaxis.line.set_visible(False) 

Using matplotlib backend: TkAgg


/tmp/ipykernel_3233/1188419287.py:29: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax_full.w_xaxis.pane.fill = False
/tmp/ipykernel_3233/1188419287.py:30: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax_full.w_yaxis.pane.fill = False
/tmp/ipykernel_3233/1188419287.py:31: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_full.w_zaxis.pane.fill = True
/tmp/ipykernel_3233/1188419287.py:32: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_full.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_3233/1188419287.py:37: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis

In [10]:
reducedPosTableY = reducedMarkersTable.iloc[:,::3]
reducedPosTableZ = reducedMarkersTable.iloc[:,1::3]
reducedPosTableX = reducedMarkersTable.iloc[:,2::3]

reducedPosTableX = reducedPosTableX.rename(columns=dict(zip(list(reducedPosTableX),[name.replace('Z','X').replace('Y','X') for name in list(reducedPosTableX)])))
reducedPosTableY = reducedPosTableY.rename(columns=dict(zip(list(reducedPosTableY),[name.replace('Z','Y').replace('X','Y') for name in list(reducedPosTableY)])))
reducedPosTableZ = reducedPosTableZ.rename(columns=dict(zip(list(reducedPosTableZ),[name.replace('X','Z').replace('Y','Z') for name in list(reducedPosTableZ)])))

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

In [11]:
fig_reduced = plt.figure(figsize=(960/100, 540/100), dpi=100)

# Add a 3D subplot with the specified position and size
ax_reduced = fig_reduced.add_axes([-0.3, -0.2, 1.6, 1.6], projection='3d')
ax_reduced.view_init(elev=10, azim=180)


reducedMinMax = np.zeros((2,3))
reducedMinMax[0,:] = [reducedPosTableX.values.min(),reducedPosTableY.values.min(),reducedPosTableZ.values.min()]
reducedMinMax[1,:] = [reducedPosTableX.values.max(),reducedPosTableY.values.max(),reducedPosTableZ.values.max()]

# Set appropriate axis limits
ax_reduced.set_xlim([reducedMinMax[0,0],reducedMinMax[1,0]])
ax_reduced.set_ylim([reducedMinMax[0,1],reducedMinMax[1,1]])
ax_reduced.set_zlim([reducedMinMax[0,2],reducedMinMax[1,2]])


plt.rcParams['grid.color'] = 'white'
ax_reduced.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax_reduced.w_xaxis.pane.fill = False
ax_reduced.w_yaxis.pane.fill = False
ax_reduced.w_zaxis.pane.fill = True
ax_reduced.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax_reduced.xaxis.pane.set_edgecolor('w')
ax_reduced.yaxis.pane.set_edgecolor('w')
ax_reduced.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax_reduced.w_xaxis.line.set_visible(False)
ax_reduced.w_yaxis.line.set_visible(True)
ax_reduced.w_zaxis.line.set_visible(False) 

/tmp/ipykernel_3233/958718550.py:21: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax_reduced.w_xaxis.pane.fill = False
/tmp/ipykernel_3233/958718550.py:22: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax_reduced.w_yaxis.pane.fill = False
/tmp/ipykernel_3233/958718550.py:23: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_reduced.w_zaxis.pane.fill = True
/tmp/ipykernel_3233/958718550.py:24: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_reduced.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_3233/958718550.py:29: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Us

In [12]:

# Function to update the plot based on the slider value
def get_frame_full_skeleton(fig, ax, val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = fullPosTableX.iloc[val]
    filteredY = fullPosTableY.iloc[val]
    filteredZ = fullPosTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=10)

    # Add edges based on the weight matrix
    for i in range(len(fullAdjacencyMatrix)):
        for j in range(i + 1, len(fullAdjacencyMatrix)):
            if fullAdjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='k', linestyle='-', linewidth=1)

    ax.set_xlim([0,1])
    ax.set_ylim([0,1])
    ax.set_zlim([fullMinMax[0,2],fullMinMax[1,2]])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    frame = np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

    return frame

def get_frame_reduced_skeleton(fig, ax, val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = reducedPosTableX.iloc[val]
    filteredY = reducedPosTableY.iloc[val]
    filteredZ = reducedPosTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=50)

    # Add edges based on the weight matrix
    for i in range(len(adjacencyMatrix)):
        for j in range(i + 1, len(adjacencyMatrix)):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='k', linestyle='-', linewidth=1.5)

    ax.set_xlim([0,1])
    ax.set_ylim([0,1])
    ax.set_zlim([reducedMinMax[0,2],reducedMinMax[1,2]])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    frame = np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)

    return frame

# Creating video of the VidCap / 64 Markers Skeleton / 20 Markers Skeleton

In [13]:
import cv2
from tqdm.notebook import tqdm

# Create a VideoWriter object in OpenCV
full_hd = (1920,1080)
record = not os.path.isfile(video_path[:-4]+'_'+str(fps * 2)+'fps_plus_skeleton.mp4')
if record:
    out = cv2.VideoWriter(video_path[:-4]+'_'+str(fps * 2)+'fps_plus_skeleton.mp4', 
                          cv2.VideoWriter_fourcc(*'mp4v'), 
                          fps*2, 
                          full_hd)  # Adjust the filename, codec, frame rate, and frame size

vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(fps * 2)+'fps_540p.mp4')
#print(vidcap.get(cv2.CAP_PROP_FRAME_COUNT),len(fullMarkersTable))
filler_size = tuple(map(int,(vidcap.get(4),(full_hd[0]-vidcap.get(3))/2,3)))
filler = np.zeros(filler_size,dtype=np.uint8)+255
# Loop to update the plot and write frames to the video
for val in tqdm(range(len(fullMarkersTable)*(int(record)*2-1)),'Saving video',miniters=20):  # Replace total_frames with the actual number of frames
    ret, video_frame = vidcap.read()
    if not ret:
        break    
    # Draw an empty blue rectangle
    video_frame = cv2.rectangle(video_frame, (10, 10), (int(vidcap.get(3)-11), int(vidcap.get(4)-11)), (0, 0, 255), 2)
    video_frame = cv2.rectangle(video_frame, (0, 0), (int(vidcap.get(3)-2), int(vidcap.get(4)-2)), (127, 127, 127), 3)
    frame_full_skeleton = cv2.cvtColor(get_frame_full_skeleton(fig_full, ax_full, val), cv2.COLOR_RGB2BGR)
    frame_reduced_skeleton = cv2.cvtColor(get_frame_reduced_skeleton(fig_reduced, ax_reduced, val), cv2.COLOR_RGB2BGR)
    out.write(np.vstack((np.hstack((filler,video_frame,filler)), np.hstack((frame_full_skeleton,frame_reduced_skeleton)))))


# Release the VideoWriter and close the OpenCV window (if used)
if record:
    out.release()
vidcap.release()

Saving video: 0it [00:00, ?it/s]

# Creating the version VidCap / 20 Markers clustered / 20 Markers Clustered stabilized

In [14]:
reducedPosTableY = reducedMarkersTable.iloc[:,::3]
reducedPosTableZ = reducedMarkersTable.iloc[:,1::3]
reducedPosTableX = reducedMarkersTable.iloc[:,2::3]

reducedPosTableX = reducedPosTableX.rename(columns=dict(zip(list(reducedPosTableX),[name.replace('Z','X').replace('Y','X') for name in list(reducedPosTableX)])))
reducedPosTableY = reducedPosTableY.rename(columns=dict(zip(list(reducedPosTableY),[name.replace('Z','Y').replace('X','Y') for name in list(reducedPosTableY)])))
reducedPosTableZ = reducedPosTableZ.rename(columns=dict(zip(list(reducedPosTableZ),[name.replace('X','Z').replace('Y','Z') for name in list(reducedPosTableZ)])))

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

In [15]:
plt.close('all')
fig_reduced = plt.figure(figsize=(960/100, 720/100), dpi=100)

# Add a 3D subplot with the specified position and size
ax_reduced = fig_reduced.add_axes([-0.3, -0.2, 1.6, 1.6], projection='3d')
ax_reduced.view_init(elev=10, azim=180)


reducedMinMax = np.zeros((2,3))
reducedMinMax[0,:] = [reducedPosTableX.values.min(),reducedPosTableY.values.min(),reducedPosTableZ.values.min()]
reducedMinMax[1,:] = [reducedPosTableX.values.max(),reducedPosTableY.values.max(),reducedPosTableZ.values.max()]

# Set appropriate axis limits
ax_reduced.set_xlim([reducedMinMax[0,0],reducedMinMax[1,0]])
ax_reduced.set_ylim([reducedMinMax[0,1],reducedMinMax[1,1]])
ax_reduced.set_zlim([reducedMinMax[0,2],reducedMinMax[1,2]])


plt.rcParams['grid.color'] = 'white'
ax_reduced.tick_params(axis='both', colors='w')
# Remove box faces except the ground
ax_reduced.w_xaxis.pane.fill = False
ax_reduced.w_yaxis.pane.fill = False
ax_reduced.w_zaxis.pane.fill = True
ax_reduced.w_zaxis.pane.set_facecolor((0.92,.92,.92))
ax_reduced.xaxis.pane.set_edgecolor('w')
ax_reduced.yaxis.pane.set_edgecolor('w')
ax_reduced.zaxis.pane.set_edgecolor((0,0,0))
# Hide axis spines and ticks
ax_reduced.w_xaxis.line.set_visible(False)
ax_reduced.w_yaxis.line.set_visible(True)
ax_reduced.w_zaxis.line.set_visible(False) 

/tmp/ipykernel_3233/253775057.py:22: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use xaxis instead.
  ax_reduced.w_xaxis.pane.fill = False
/tmp/ipykernel_3233/253775057.py:23: MatplotlibDeprecationWarning: The w_yaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use yaxis instead.
  ax_reduced.w_yaxis.pane.fill = False
/tmp/ipykernel_3233/253775057.py:24: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_reduced.w_zaxis.pane.fill = True
/tmp/ipykernel_3233/253775057.py:25: MatplotlibDeprecationWarning: The w_zaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Use zaxis instead.
  ax_reduced.w_zaxis.pane.set_facecolor((0.92,.92,.92))
/tmp/ipykernel_3233/253775057.py:30: MatplotlibDeprecationWarning: The w_xaxis attribute was deprecated in Matplotlib 3.1 and will be removed in 3.8. Us

In [25]:
import cv2
color_pool = ['#e31a1c', '#1f78b4', '#33a02c', '#ff7f00', '#6a3d9a', '#b15928']

def get_frame_reduced_skeleton_clustered(fig, ax, val, clusters,subtitle):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = reducedPosTableX.iloc[val]
    filteredY = reducedPosTableY.iloc[val]
    filteredZ = reducedPosTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=100,c=[color_pool[i] for i in clusters])

    # Add edges based on the weight matrix
    for i in range(len(adjacencyMatrix)):
        for j in range(i + 1, len(adjacencyMatrix)):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='k', linestyle='-', linewidth=1.5)

    ax.set_xlim([0,1])
    ax.set_ylim([0,1])
    ax.set_zlim([reducedMinMax[0,2],reducedMinMax[1,2]])

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    ncols, nrows = fig.canvas.get_width_height()
    frame = np.frombuffer(buf, dtype=np.uint8).reshape(nrows, ncols, 3)
    frame = cv2.putText(frame, subtitle, ((frame.shape[1]-cv2.getTextSize(subtitle, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0][0])//2, frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    return frame

In [17]:
from clustering import table_to_list_xyz_tables, group_table_by_joints, compute_derivatives, xyz_tables_to_xyz_columns, smoothing, joints_array_to_xyz_columns
import re

# Calculate angular momentum
if not os.path.isfile('_angmom.csv'): 
    dt = ((float(endSec)-float(startSec)) / len(reducedMarkersTable))
    centerOfMassPosTable = pd.concat([table.apply(lambda row: row.sum()/20,axis=1) for table in table_to_list_xyz_tables(reducedMarkersTable)],axis=1)
    relativePosTable = group_table_by_joints(xyz_tables_to_xyz_columns([posTableAxis.subtract(centerOfMassPosTable.iloc[:,j],axis=0) for j,posTableAxis in enumerate(table_to_list_xyz_tables(reducedMarkersTable))]))
    centerOfMassVelocityTable = compute_derivatives(centerOfMassPosTable,dt,smooth=False)
    relativeVelocityTable = group_table_by_joints(xyz_tables_to_xyz_columns([velocityTableAxis.subtract(centerOfMassVelocityTable.iloc[:,j],axis=0) for j,velocityTableAxis in enumerate(table_to_list_xyz_tables(compute_derivatives(reducedMarkersTable,dt)))]))
    angularMomentumTable = relativePosTable
    for i in range(len(relativePosTable)):
        for j in range(len(relativePosTable.columns)):
            angularMomentumTable.iloc[i,j][:] = np.cross(relativePosTable.iloc[i,j],relativeVelocityTable.iloc[i,j])
    angularMomentumTable = group_table_by_joints(smoothing(joints_array_to_xyz_columns(angularMomentumTable)))
    # Soooo long to recompute each time... better save a file
    angularMomentumTable.to_csv('_angmom.csv', index=False)
else:
    angularMomentumTable = pd.read_csv('_angmom.csv')
    angularMomentumTable = angularMomentumTable.applymap(lambda cell: np.array(eval(re.sub(r'(?<=[\d-])\s{1,}(?=[\d-])', ', ',cell))))


In [18]:
import cv2
from tqdm.notebook import tqdm
import os
from clustering import *

# Create a VideoWriter object in OpenCV
full_hd = (1920,1080)
record = not os.path.isfile(video_path[:-4]+'_'+str(fps * 2)+'fps_clustered.mp4')
if record:
    out = cv2.VideoWriter(video_path[:-4]+'_'+str(fps * 2)+'fps_clustered.mp4', 
                          cv2.VideoWriter_fourcc(*'mp4v'), 
                          fps*2, 
                          full_hd)  # Adjust the filename, codec, frame rate, and frame size

vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(fps * 2)+'fps_360p.mp4')
filler_size = tuple(map(int,(vidcap.get(4),(full_hd[0]-vidcap.get(3))/2,3)))
filler = np.zeros(filler_size,dtype=np.uint8)+255
weightMatrix = calculate_weight_matrix(angularMomentumTable.iloc[0],adjacencyMatrix)
stabilized_labels = shi_malik_spectral_clustering_matlab_version(weightMatrix)
# Loop to update the plot and write frames to the video
for val in tqdm(range(len(fullMarkersTable)*(int(record)*2-1)),'Saving video',miniters=20):  # Replace total_frames with the actual number of frames
    ret, video_frame = vidcap.read()
    if not ret:
        break    
    weightMatrix = calculate_weight_matrix(angularMomentumTable.iloc[val],adjacencyMatrix)
    predicted_labels = shi_malik_spectral_clustering_matlab_version(weightMatrix)
    stabilized_labels = compute_minimum_weight_cluster(stabilized_labels,predicted_labels,'MWPM')
    # Draw an empty blue rectangle
    video_frame = cv2.rectangle(video_frame, (10, 10), (int(vidcap.get(3)-11), int(vidcap.get(4)-11)), (0, 0, 255), 2)
    video_frame = cv2.rectangle(video_frame, (0, 0), (int(vidcap.get(3)-2), int(vidcap.get(4)-2)), (127, 127, 127), 3)
    frame_clustered = cv2.cvtColor(get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val, predicted_labels,'Simple Clustering'), cv2.COLOR_RGB2BGR)
    frame_clustered_stabilized = cv2.cvtColor(get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val, stabilized_labels,'Stabilized Clustering'), cv2.COLOR_RGB2BGR)
    out.write(np.vstack((np.hstack((filler,video_frame,filler)), np.hstack((frame_clustered,frame_clustered_stabilized)))))

# Release the VideoWriter and close the OpenCV window (if used)
if record:
    out.release()
vidcap.release()
#plt.close('all')

Saving video: 0it [00:00, ?it/s]

In [19]:
import networkx as nx

skeleton_graph = nx.from_numpy_array(adjacencyMatrix)
leaf_nodes = [node for node in skeleton_graph.nodes() if skeleton_graph.degree(node) == 1]

def smooth_clusters_VBase(previous_clusters:list, current_clusters:list,weightMatrix,i:int=0, delay:int=-1):
    if delay <= 0 or i % delay != 0:
        return previous_clusters

    # Compute the frontier of the previous clusters for every cluster
    frontier_nodes = { cluster_id: {
                        neighbor
                        for node in nodes
                        for neighbor in skeleton_graph.neighbors(node)
                        if neighbor not in nodes
                        }
                        for cluster_id, nodes in {key: set([i for i, val in enumerate(previous_clusters) if val == key]) 
                                                    for key in set(previous_clusters)}.items()
                     }
    
    # Update only clusters whose nodes are at the frontier of each cluster
    new_clusters = { node: cluster_id
                     for cluster_id in set(previous_clusters)
                     for node in frontier_nodes[cluster_id]
                     if current_clusters[node] == cluster_id }

    # Add remaining nodes and update the leaf nodes (to avoid cluster disappearing)
    new_clusters.update({ node: current_clusters[node] if not node in new_clusters else cluster_id
                            for node, cluster_id in enumerate(previous_clusters) })
    
    # TODO further improvement may implement the possibility to choose the node from which the
    # cluster should fade in, based on the weakest similarity value
    return [cluster_id for _,cluster_id in sorted(new_clusters.items())]

In [20]:
import networkx as nx

skeleton_graph = nx.from_numpy_array(adjacencyMatrix)
leaf_nodes = [node for node in skeleton_graph.nodes() if skeleton_graph.degree(node) == 1]

def smooth_clusters(previous_clusters: list, current_clusters: list, weightMatrix, i: int = 0, delay: int = -1):
    if delay <= 0 or i % delay != 0:
        return previous_clusters

    # Step 1: Create dictionaries for current and previous clusters
    current_clusters_dict = {cluster_id: [node for node, cluster in enumerate(current_clusters) if cluster == cluster_id] for cluster_id in set(current_clusters)}
    previous_clusters_dict = {cluster_id: [node for node, cluster in enumerate(previous_clusters) if cluster == cluster_id] for cluster_id in set(previous_clusters)}

    # Step 2: Subtract current cluster dictionary from previous clusters
    changed_clusters = {cluster_id: list(set(current_clusters_dict[cluster_id]) - set(previous_clusters_dict.get(cluster_id, [])) ) for cluster_id in current_clusters_dict}

    # Step 3: Sort nodes by increasing weighted degree centrality, grouped by cluster
    sorted_nodes_by_centrality = {
        cluster_id: [node for node, _ in sorted({node: sum(weightMatrix[node][neighbor] for neighbor in nodes) 
                                                 for node in nodes}.items(), key=lambda x: x[1], reverse=True)]
        for cluster_id, nodes in changed_clusters.items()}

    # Step 4: Update one node with the lowest weighted degree centrality for each cluster
    out_clusters = {sorted_nodes[0]: cluster_id for cluster_id, sorted_nodes in sorted_nodes_by_centrality.items() if sorted_nodes}


    out_clusters.update({ node: cluster_id for node, cluster_id in enumerate(previous_clusters) if not node in out_clusters })

    return [cluster_id for _,cluster_id in sorted(out_clusters.items())]


In [21]:
import cv2
from tqdm.notebook import tqdm
import os
from clustering import *
import warnings
warnings.filterwarnings("ignore", category=Warning)

# Create a VideoWriter object in OpenCV
full_hd = (1920,1080)
record = not os.path.isfile(video_path[:-4]+'_'+str(fps * 2)+'fps_clustered_delayed.mp4')
if record:
    out = cv2.VideoWriter(video_path[:-4]+'_'+str(fps * 2)+'fps_clustered_delayed.mp4', 
                          cv2.VideoWriter_fourcc(*'mp4v'), 
                          fps*2, 
                          full_hd)  # Adjust the filename, codec, frame rate, and frame size

vidcap = cv2.VideoCapture(video_path[:-4]+'_'+str(fps * 2)+'fps_360p.mp4')
filler_size = tuple(map(int,(vidcap.get(4),(full_hd[0]-vidcap.get(3))/2,3)))
filler = np.zeros(filler_size,dtype=np.uint8)+255
weightMatrix = calculate_weight_matrix(angularMomentumTable.iloc[0],adjacencyMatrix)
stabilized_labels = shi_malik_spectral_clustering_matlab_version(weightMatrix)
stabilized_smoothed_labels = stabilized_labels

# Loop to update the plot and write frames to the video
for val in tqdm(range(len(fullMarkersTable)*(int(record)*2-1)),'Saving video',miniters=20):  # Replace total_frames with the actual number of frames
    ret, video_frame = vidcap.read()
    if not ret:
        break    
    weightMatrix = calculate_weight_matrix(angularMomentumTable.iloc[val],adjacencyMatrix)
    predicted_labels = shi_malik_spectral_clustering_matlab_version(weightMatrix)
    stabilized_labels = compute_minimum_weight_cluster(stabilized_labels,predicted_labels,'MWPM')
    stabilized_smoothed_labels = smooth_clusters(stabilized_smoothed_labels,stabilized_labels,weightMatrix,val,fps//4)

    # Draw an empty blue rectangle
    video_frame = cv2.rectangle(video_frame, (10, 10), (int(vidcap.get(3)-11), int(vidcap.get(4)-11)), (0, 0, 255), 2)
    video_frame = cv2.rectangle(video_frame, (0, 0), (int(vidcap.get(3)-2), int(vidcap.get(4)-2)), (127, 127, 127), 3)
    frame_clustered = cv2.cvtColor(get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val, stabilized_labels,'Stabilized Clustering'), cv2.COLOR_RGB2BGR)
    frame_clustered_stabilized = cv2.cvtColor(get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val, stabilized_smoothed_labels,'Stabilized Smoothed'), cv2.COLOR_RGB2BGR)
    #frame_clustered_stabilized = cv2.putText(frame_clustered_stabilized, str(val), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    out.write(np.vstack((np.hstack((filler,video_frame,filler)), np.hstack((frame_clustered,frame_clustered_stabilized)))))

# Release the VideoWriter and close the OpenCV window (if used)
if record:
    out.release()
vidcap.release()

Saving video: 0it [00:00, ?it/s]

In [30]:
val = 704
next = 3


weightMatrix = calculate_weight_matrix(angularMomentumTable.iloc[val],adjacencyMatrix)
standard_labels = shi_malik_spectral_clustering_matlab_version(weightMatrix)
plt.imsave('_imgs/pure/pure_t.png',get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val, standard_labels,'t = '+str(val)[0]+'.'+str(val)[1:]))
stabilized_smoothed_labels = standard_labels
stabilized_labels = standard_labels
for i in range(30):
    weightMatrix = calculate_weight_matrix(angularMomentumTable.iloc[val],adjacencyMatrix)
    standard_labels = shi_malik_spectral_clustering_matlab_version(weightMatrix)
    weightMatrix = calculate_weight_matrix(angularMomentumTable.iloc[val+next],adjacencyMatrix)
    standard_labels_next = shi_malik_spectral_clustering_matlab_version(weightMatrix)
    stabilized_labels = compute_minimum_weight_cluster(stabilized_labels,standard_labels_next,'MWPM')

    plt.imsave('_imgs/pure/pure_next_'+str(i)+'.png',get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val+next, standard_labels_next,'t = '+str(val+next)[0]+'.'+str(val+next)[1:]))
    #print(standard_labels)
    #print(standard_labels_next)
    #print(compute_minimum_weight_cluster(standard_labels,standard_labels_next,'MWPM'))
    #print(list(stabilized_smoothed_labels))
    plt.imsave('_imgs/stabilized/stabilized_next'+str(i)+'.png',get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val+next, stabilized_labels,'t = '+str(val+next)[0]+'.'+str(val+next)[1:]))

    stabilized_smoothed_labels = smooth_clusters(stabilized_smoothed_labels,stabilized_labels,weightMatrix,i,2)
    plt.imsave('_imgs/smoothed/smoothed_next'+str(i)+'.png',get_frame_reduced_skeleton_clustered(fig_reduced, ax_reduced, val+next, stabilized_smoothed_labels,'t = '+str(val+next)[0]+'.'+str(val+next)[1:]))
    val += next


In [31]:
from PIL import Image
import os

# List of image files
image_files = []  # Replace with your file names
for i in range(30):
    image_files.append('_imgs/smoothed/smoothed_next'+str(i)+'.png')

# Create a list to store the images
images = []

# Load each image and append it to the list
for image_file in image_files:
    img = Image.open(image_file)
    images.append(img)


# Set the path for the output GIF
output_gif = 'output.gif'

# Save the list of images as a GIF
images[0].save(output_gif, format='GIF', save_all=True, append_images=images[1:],duration=1,loop=0,quality=20)

# Clean up (optional): delete the individual image files if needed
#for image_file in image_files:
#    os.remove(image_file)

print(f'GIF saved as {output_gif}')

GIF saved as output.gif
